In [1]:
from plotbee.video import Video
from plotbee.utils import save_json
import numpy as np

In [24]:
# video = Video.load("/mnt/storage/work/jchan/final_beepose/one_week_data/tags_skeleton_merged_C02_170622120000_fine_new.json")
# "/mnt/storage/work/jchan/gurabo10/detections/col02/hungarian_tag_10_02_R_190717140000_skeleton.json"
# video = Video.load("/mnt/storage/work/jchan/gurabo10/detections/col08/hungarian_tag_10_02_R_190718140000_skeleton.json")
video = Video.load("/mnt/storage/work/jchan/gurabo10/detections/col02/hungarian_tag_10_02_R_190717140000_skeleton.json")

100%|██████████| 72007/72007 [00:27<00:00, 2631.27it/s] 


In [25]:
video[1000][1].__dict__

{'_parts': {1: [(2152, 284)],
  2: [(2152, 480)],
  3: [(2152, 336)],
  4: [(2136, 712)]},
 '_center_part': 3,
 '_connections': [[1, 3], [3, 2], [2, 4]],
 '_frame': Frame: 1000
 Body(id=272, parts={1: [(1472, 220)], 2: [(1480, 416)], 3: [(1476, 284)], 4: [(1504, 608)])
 Body(id=266, parts={1: [(2152, 284)], 2: [(2152, 480)], 3: [(2152, 336)], 4: [(2136, 712)])
 Body(id=273, parts={1: [(1544, 632)], 2: [(1612, 836)], 3: [(1572, 700)], 4: [(1620, 1012)])
 Body(id=251, parts={1: [(1980, 428)], 2: [(1788, 340)], 3: [(1920, 404)], 4: [(1576, 316)])
 Body(id=268, parts={1: [(720, 928)], 2: [(676, 756)], 3: [(700, 868)], 4: [(568, 516)])
 Body(id=265, parts={1: [(600, 892)], 2: [(508, 1088)], 3: [(568, 956)], 4: [(468, 1248)])
 Body(id=276, parts={1: [(1880, 676)], 2: [(1856, 884)], 3: [(1872, 740)], 4: [(1824, 1080)]),
 '_id': 266,
 '_angle_conn': (1, 3),
 '_prev': Body(id=266, parts={1: [(2112, 296)], 2: [(2116, 508)], 3: [(2112, 360)], 4: [(2128, 736)]),
 '_next': Body(id=266, parts={1: [(

In [14]:
video.load_video("/mnt/storage/Gurabo/datasets/gurabo10/mp4/col08/10_02_R_190718140000.mp4")

In [15]:
video.hungarian_tracking(max_dist=150)

  0%|          | 42/71984 [00:00<02:51, 419.82it/s]

Tracking...


100%|██████████| 71984/71984 [00:46<00:00, 1546.90it/s]


Convert to Track format


In [23]:
video[1000][1].angle

50.59933933652057

In [20]:
def convert_labelbee(video):
    labelbee_data = dict()
    labelbee_data["info"] = {"type":"events-multiframe",
                             "source":"Converted from Tracks v1 object"}
    labelbee_data["data"] = dict()
    for frame in video:
        frame_anno = extract_frame_annotations(frame)
        labelbee_data["data"][frame.id] = frame_anno
    return labelbee_data

def extract_frame_annotations(frame):
    bodies_anno = list()
    for body in frame:
        body_anno = extract_body_annotation(body)
        bodies_anno.append(body_anno)
    return bodies_anno

def extract_body_annotation(body):
    entry = dict()
    entry["id"] = body.id
    entry["time"] = body.frameid/20
    entry["frame"] = body.frameid
    x, y = body.center
    entry["x"] = x - 136.867/2
    entry["y"] = y - 212.324/2
    entry["cx"] = x - 136.867/2
    entry["cy"] = y - 212.324/2
    entry["width"] = 136.867
    entry["height"] = 212.324
    entry["angle"] = -body.angle + 180.0
    entry["notes"] = ""
    entry["labels"]= ""
    parts = list()
    for pid, keypoints in body.parts.items():
        x, y = keypoints[0]
        parts.append({"posFrame":{"x": x, "y":y}, "label":str(pid)})
    entry["parts"] = parts
    return entry

In [21]:
short_video = video[:12000]
labelbee_data = convert_labelbee(short_video)

In [22]:
save_json("10_02_R_190718140000_labelbee_0_12000.json", labelbee_data)